# 02_baseline_tfidf — Binario A/D

**Objetivo:** baseline clásico **robusto a ruido** (typos/transcripción) con **char TF‑IDF (3–5)** + **SVM (LinearSVC)**.  
**Justificación:** los n‑gramas de caracteres capturan patrones ortográficos aun con errores; es un buen contrapunto al enfoque rule‑based.


In [18]:
# ===============================================================
# Setup: Paths, Imports, y Utilidades Compartidas
# ===============================================================

from pathlib import Path
import pandas as pd
import re, unicodedata, os

# Intentar importar utilidades compartidas
try:
    import sys
    sys.path.insert(0, str(Path.cwd()))
    from utils_shared import setup_paths, guess_text_col, guess_label_col, normalize_label
    print("[INFO] Utilizando utils_shared.py")
    
    # Setup de paths centralizado
    paths = setup_paths()
    BASE_PATH = paths['BASE_PATH']
    DATA_PATH = paths['DATA_PATH']
    SPLITS_PATH = paths['SPLITS_PATH']
    
    # Usar funciones centralizadas
    _guess_text_col = guess_text_col
    _guess_label_col = guess_label_col
    _norm_label_bin = normalize_label
    
except ImportError:
    print("[WARNING] utils_shared.py no encontrado, usando funciones locales")
    
    # Setup manual de paths
    BASE_PATH = Path.cwd()
    if BASE_PATH.name == "notebooks":
        BASE_PATH = BASE_PATH.parent
    
    DATA_PATH = BASE_PATH / "data"
    SPLITS_PATH = DATA_PATH / "splits"
    
    DATA_PATH.mkdir(exist_ok=True)
    
    # Funciones helper locales
    def _guess_text_col(df):
        for c in ["texto", "text", "comment", "comentario"]:
            if c in df.columns:
                return c
        return df.columns[0]
    
    def _guess_label_col(df):
        for c in ["etiqueta", "label", "category"]:
            if c in df.columns:
                return c
        return df.columns[1] if len(df.columns) > 1 else df.columns[-1]
    
    def _norm_label_bin(s):
        if pd.isna(s): 
            return ""
        s = str(s).strip().lower()
        s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
        return {'depresivo': 'depresion'}.get(s, s)

# Validar existencia de splits
if not SPLITS_PATH.exists():
    raise FileNotFoundError(
        f"[ERROR] Splits no encontrados en {SPLITS_PATH}\n"
        f"        Debes ejecutar primero: 02_create_splits.ipynb"
    )

print("Paths configurados:")
print(f"  BASE_PATH:   {BASE_PATH}")
print(f"  DATA_PATH:   {DATA_PATH}")
print(f"  SPLITS_PATH: {SPLITS_PATH}")

# Columnas esperadas en dataset_base.csv
TEXT_COL = "texto"
LABEL_COL = "etiqueta"

[INFO] Utilizando utils_shared.py
Paths configurados:
  BASE_PATH:   /Users/manuelnunez/Projects/psych-phenotyping-paraguay
  DATA_PATH:   /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data
  SPLITS_PATH: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/splits


## 1) Carga y preprocesamiento **agresivo** (pensado para ML clásico)

In [19]:
# Carga de datos y preprocesamiento agresivo para TF-IDF
# 
# Estrategia: Máxima normalización (lowercase, sin tildes, sin símbolos)
# - Reduce vocabulario y mejora robustez ante typos
# - Marca negaciones: "no tengo" → "no_tengo" (feature diferenciado)
# - Char n-grams (3-5): robustos a variantes ortográficas
#
# Comparación: Rule-based conserva tildes, BETO solo tokeniza, TF-IDF normaliza todo

import pandas as pd, re, unicodedata

# Cargar splits unificados desde 02_create_splits.ipynb
dataset_base = pd.read_csv(SPLITS_PATH / 'dataset_base.csv')
train_indices = pd.read_csv(SPLITS_PATH / 'train_indices.csv')['row_id'].values
val_indices = pd.read_csv(SPLITS_PATH / 'val_indices.csv')['row_id'].values

text_col = _guess_text_col(dataset_base)
label_col = _guess_label_col(dataset_base)

print(f"[INFO] Splits: {len(train_indices)} train, {len(val_indices)} val")

# Definir función de limpieza agresiva
RE_MULTI = re.compile(r'(.)\1{2,}')  # Detecta 3+ letras repetidas

def clean_text_ml(s: str) -> str:
    """
    Limpieza AGRESIVA para TF-IDF (máxima normalización para robustez).
    
    Aplica:
    - Lowercase completo
    - Normalización NFC + eliminación de tildes
    - Colapso de alargamientos
    - Eliminación de símbolos especiales (preserva puntuación básica)
    - Marca negaciones simples ("no X" → "no_X")
    """
    if pd.isna(s):
        return ""
    
    s = str(s).lower().strip()           # Lowercase total
    s = unicodedata.normalize("NFC", s)  # Normaliza tildes
    s = RE_MULTI.sub(r'\1\1', s)         # Colapsa alargamientos
    
    # Elimina símbolos especiales (preserva letras, números, puntuación básica)
    s = re.sub(r"[^a-z0-9áéíóúüñ\s.,!?:/\-]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()   # Normaliza espacios
    
    # Marca negaciones simples: "no tengo" → "no_tengo"
    s = re.sub(r"\bno\s+([a-záéíóúüñ]{2,})", r"no_\1", s)
    
    return s

dataset_base['texto_ml'] = dataset_base[text_col].map(clean_text_ml)

df_train = dataset_base[dataset_base['row_id'].isin(train_indices)].copy()
df_val = dataset_base[dataset_base['row_id'].isin(val_indices)].copy()

X_train, y_train = df_train['texto_ml'], df_train[label_col]
X_val, y_val = df_val['texto_ml'], df_val[label_col]

print(f"[INFO] Distribución train: {dict(y_train.value_counts())}")
print(f"[INFO] Distribución val: {dict(y_val.value_counts())}")

[INFO] Splits: 2500 train, 626 val
[INFO] Distribución train: {'depresion': 1760, 'ansiedad': 740}
[INFO] Distribución val: {'depresion': 441, 'ansiedad': 185}
[INFO] Distribución train: {'depresion': 1760, 'ansiedad': 740}
[INFO] Distribución val: {'depresion': 441, 'ansiedad': 185}


## 2) Split estratificado y entrenamiento (char TF‑IDF + LinearSVC)

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

tfidf_char = TfidfVectorizer(
    analyzer='char_wb',
    ngram_range=(3,5),
    min_df=2,
    max_df=0.95
)

clf = Pipeline([
    ('tfidf', tfidf_char),
    ('svm', LinearSVC(class_weight='balanced', random_state=42))
])

clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)
print("[INFO] Entrenamiento completado.")

[INFO] Entrenamiento completado.


## 3) Métricas y exportables

In [21]:
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, confusion_matrix

tfidf_pred_csv   = DATA_PATH/'tfidf_predictions.csv'
tfidf_report_csv = DATA_PATH/'tfidf_classification_report.csv'
tfidf_eval_csv   = DATA_PATH/'tfidf_eval.csv'
tfidf_cm_csv     = DATA_PATH/'tfidf_confusion_matrix.csv'

classes = ['depresion','ansiedad']

pd.DataFrame(classification_report(y_val, y_pred, labels=classes, output_dict=True, zero_division=0))  .transpose().to_csv(tfidf_report_csv, index=True, encoding='utf-8')

pd.DataFrame([{
    'macro_f1': f1_score(y_val, y_pred, average='macro', zero_division=0),
    'macro_precision': precision_score(y_val, y_pred, average='macro', zero_division=0),
    'macro_recall': recall_score(y_val, y_pred, average='macro', zero_division=0),
    'n': int(len(y_val))
}]).to_csv(tfidf_eval_csv, index=False, encoding='utf-8')

cm = confusion_matrix(y_val, y_pred, labels=classes)
pd.DataFrame(cm, index=[f'true_{c}' for c in classes], columns=[f'pred_{c}' for c in classes]).to_csv(tfidf_cm_csv)

pd.DataFrame({'texto': X_val, 'y_true': y_val, 'y_pred': y_pred}).to_csv(tfidf_pred_csv, index=False, encoding='utf-8')

print("[INFO] Exportados:")
print("  - Predicciones:", tfidf_pred_csv)
print("  - Reporte:", tfidf_report_csv)
print("  - Métricas:", tfidf_eval_csv)
print("  - Matriz:", tfidf_cm_csv)

[INFO] Exportados:
  - Predicciones: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/tfidf_predictions.csv
  - Reporte: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/tfidf_classification_report.csv
  - Métricas: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/tfidf_eval.csv
  - Matriz: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/tfidf_confusion_matrix.csv


## 4) Análisis de Errores (FP/FN)

In [22]:
# Exportar errores para análisis cualitativo
# FP = predijo X cuando era Y (falsos positivos de cada clase)
# FN = predijo Y cuando era X (falsos negativos de cada clase)

# False Positives: casos predichos como depresión cuando eran ansiedad
fp_depresion = df_val[(y_val == 'ansiedad') & (y_pred == 'depresion')].copy()
fp_depresion['error_type'] = 'FP_depresion'

# False Negatives: casos que eran depresión pero se predijeron como ansiedad
fn_depresion = df_val[(y_val == 'depresion') & (y_pred == 'ansiedad')].copy()
fn_depresion['error_type'] = 'FN_depresion'

# False Positives: casos predichos como ansiedad cuando eran depresión
fp_ansiedad = df_val[(y_val == 'depresion') & (y_pred == 'ansiedad')].copy()
fp_ansiedad['error_type'] = 'FP_ansiedad'

# False Negatives: casos que eran ansiedad pero se predijeron como depresión  
fn_ansiedad = df_val[(y_val == 'ansiedad') & (y_pred == 'depresion')].copy()
fn_ansiedad['error_type'] = 'FN_ansiedad'

# Guardar análisis de errores
tfidf_fp_dep_csv = DATA_PATH / 'tfidf_fp_depresion.csv'
tfidf_fn_dep_csv = DATA_PATH / 'tfidf_fn_depresion.csv'
tfidf_fp_ans_csv = DATA_PATH / 'tfidf_fp_ansiedad.csv'
tfidf_fn_ans_csv = DATA_PATH / 'tfidf_fn_ansiedad.csv'

fp_depresion[[text_col, label_col, 'error_type']].to_csv(tfidf_fp_dep_csv, index=False, encoding='utf-8')
fn_depresion[[text_col, label_col, 'error_type']].to_csv(tfidf_fn_dep_csv, index=False, encoding='utf-8')
fp_ansiedad[[text_col, label_col, 'error_type']].to_csv(tfidf_fp_ans_csv, index=False, encoding='utf-8')
fn_ansiedad[[text_col, label_col, 'error_type']].to_csv(tfidf_fn_ans_csv, index=False, encoding='utf-8')

print("[INFO] Análisis de errores exportado:")
print(f"  FP Depresión: {len(fp_depresion)} casos → {tfidf_fp_dep_csv.name}")
print(f"  FN Depresión: {len(fn_depresion)} casos → {tfidf_fn_dep_csv.name}")
print(f"  FP Ansiedad:  {len(fp_ansiedad)} casos → {tfidf_fp_ans_csv.name}")
print(f"  FN Ansiedad:  {len(fn_ansiedad)} casos → {tfidf_fn_ans_csv.name}")

[INFO] Análisis de errores exportado:
  FP Depresión: 24 casos → tfidf_fp_depresion.csv
  FN Depresión: 27 casos → tfidf_fn_depresion.csv
  FP Ansiedad:  27 casos → tfidf_fp_ansiedad.csv
  FN Ansiedad:  24 casos → tfidf_fn_ansiedad.csv
